### Shared Word File Process

In [14]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [15]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [17]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # 28  # native word end index

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # always must be False in this part
native_word = True # True for adding native word
etymology_word = True  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.
print(f"{file_ext}")

4


In [18]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)

### Twogram In Threegram

In [5]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
1003048,karneleme,5
1003049,karnaya,5
1003050,dörtlümüzün,5
1003051,karnavalınız,5


In [6]:
df_shared_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency{file_ext}.xlsx")
#df_shared_file = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency.xlsx")
df_shared_file

,word,frequency,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,bir,18835735.0,bir şey,859944,bir şey yok,113165.0,bir şey var mı,41773.0,komik bir şey mi var,901.0,ne var,62532.0
1,bu,11062659.0,değil mi,585879,bir şey var,110455.0,bir şey daha var,12185.0,bir şey yok değil mi,893.0,ben de,59972.0
2,ne,8025880.0,ben de,377765,bu da ne,89463.0,bir şey mi var,11834.0,o kadar çok şey var,790.0,değil mi,58386.0
3,ve,7766036.0,var mı,281902,bir şey değil,68498.0,ne var ne yok,9276.0,bir şey yok bir şey,490.0,bu da ne,50098.0
4,için,5484109.0,ne kadar,237876,bir şey mi,64933.0,bir şey değil mi,5080.0,ve bir şey daha var,393.0,ben mi,33652.0
...,...,...,...,...,...,...,...,...,...,...,...,...
17691,NaN,NaN,seni sandviç,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17692,NaN,NaN,da kariyer,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17693,NaN,NaN,buton var,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17694,NaN,NaN,bu villa,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [8]:
def word_in_wordgroup(df, list_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, list_column and target_column are 
       dataframe column string name. list_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{list_column}"].dropna():
        try:
            #word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(10)  # Option
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(100) 
        except:
            pass        
        word_in_word_cluster.insert(0,f"{list_column}",i)
        df_word_result = pd.concat([df_word_result,word_in_word_cluster], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [9]:
#df_shared_count = word_count_result(df_shared_file,["threegram"])
#df_shared_count

In [10]:
df_shared_file["twogram"].nunique()

17696

In [11]:
df_two_in_three = word_in_wordgroup(df_shared_file, "twogram", "threegram")
df_two_in_three

,twogram,threegram
0,bir şey,bir şey yok
1,bir şey,bir şey var
2,bir şey,bir şey değil
3,bir şey,bir şey mi
4,bir şey,bir şey daha
...,...,...
24336,ve fizik,biyoloji ve fizik
24337,senaryo avukat,bir senaryo avukat
24338,parti beni,bu parti beni
24339,gibi şeker,gibi şeker bir


In [12]:
df_two_in_three["twogram"].nunique()

6443

In [13]:
df_shared_select_twogram = df_shared_file.loc[:,["twogram","freq_twogram"]]
df_shared_select_twogram

,twogram,freq_twogram
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,var mı,281902
4,ne kadar,237876
...,...,...
17691,seni sandviç,6
17692,da kariyer,6
17693,buton var,6
17694,bu villa,6


In [14]:
set_shared_twogram = set(df_shared_select_twogram["twogram"])
set_two_three = set(df_two_in_three["twogram"])

In [15]:
df_twogram_in_threegram = pd.DataFrame(set_two_three, columns=["twogram"])  # columns=["twogram_in_threegram"]
df_twogram_in_threegram

,twogram
0,medyum gibi
1,kupa o
2,ama maç
3,makine o
4,de kokteyl
...,...
6438,minibüs bir
6439,şerif de
6440,ve general
6441,telefon gibi


In [16]:
df_twogram_in_threegram_freq = pd.merge(df_twogram_in_threegram, df_shared_select_twogram, how="left", on="twogram")
df_twogram_in_threegram_freq.sort_values(by="freq_twogram", ascending=False, inplace=True)
df_twogram_in_threegram_freq.rename(columns={"twogram":"twogram_in_threegram","freq_twogram":"freq_two_in_three"}, inplace=True)
df_twogram_in_threegram_freq.reset_index(drop=True, inplace=True)
df_twogram_in_threegram_freq

,twogram_in_threegram,freq_two_in_three
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,var mı,281902
4,ne kadar,237876
...,...,...
6438,psikopat değil,6
6439,için popüler,6
6440,skor var,6
6441,kampanya da,6


In [17]:
df_twogram_diff = pd.DataFrame(set_shared_twogram.difference(set_two_three), columns=["twogram"])
df_twogram_diff

,twogram
0,antrenör ben
1,metro polis
2,şeker beni
3,mi rezervasyon
4,o nükleer
...,...
11248,konsantre mi
11249,blog da
11250,jöle gibi
11251,de ağustos


In [18]:
df_twogram_diff_freq = pd.merge(df_twogram_diff, df_shared_select_twogram, how="left", on="twogram")
df_twogram_diff_freq.sort_values(by="freq_twogram", ascending=False, inplace=True)
df_twogram_diff_freq.reset_index(drop=True, inplace=True)
df_twogram_diff_freq

,twogram,freq_twogram
0,evet kaptan,1357
1,evet madam,1129
2,evet şef,946
3,ve motor,871
4,seni manyak,643
...,...,...
11248,seni sandviç,6
11249,bebek dans,6
11250,mesaj beni,6
11251,bas mı,6


In [19]:
df_shared_file["twogram"] = df_twogram_diff_freq["twogram"]
df_shared_file["freq_twogram"] = df_twogram_diff_freq["freq_twogram"]

In [20]:
df_shared_twogram_process = pd.concat([df_shared_file,df_twogram_in_threegram_freq], axis=1)
df_shared_twogram_process

,word,frequency,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence,twogram_in_threegram,freq_two_in_three
0,bir,18835735.0,evet kaptan,1357.0,bir şey yok,113165.0,bir şey var mı,41773.0,komik bir şey mi var,901.0,ne var,62532.0,bir şey,859944.0
1,bu,11062659.0,evet madam,1129.0,bir şey var,110455.0,bir şey daha var,12185.0,bir şey yok değil mi,893.0,ben de,59972.0,değil mi,585879.0
2,ne,8025880.0,evet şef,946.0,bu da ne,89463.0,bir şey mi var,11834.0,o kadar çok şey var,790.0,değil mi,58386.0,ben de,377765.0
3,ve,7766036.0,ve motor,871.0,bir şey değil,68498.0,ne var ne yok,9276.0,bir şey yok bir şey,490.0,bu da ne,50098.0,var mı,281902.0
4,için,5484109.0,seni manyak,643.0,bir şey mi,64933.0,bir şey değil mi,5080.0,ve bir şey daha var,393.0,ben mi,33652.0,ne kadar,237876.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17691,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df_shared_twogram_process.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency{file_ext}2.xlsx", index=False)

##### Concat Result With Comma

In [22]:
df_word_order_twogram = word_in_wordgroup(df_shared_twogram_process, "word", "twogram")
df_word_order_threegram = word_in_wordgroup(df_shared_twogram_process, "word", "threegram") 
df_word_order_fourgram = word_in_wordgroup(df_shared_twogram_process, "word", "fourgram") 
df_word_order_fivegram = word_in_wordgroup(df_shared_twogram_process, "word", "fivegram")
df_word_order_sentence = word_in_wordgroup(df_shared_twogram_process, "word", "sentence")
df_word_order_twogram_threegram = word_in_wordgroup(df_shared_twogram_process, "word", "twogram_in_threegram") 

In [23]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["word"])["threegram"].apply(", ".join).reset_index()
df_word_order_join_fourgram = df_word_order_fourgram.groupby(["word"])["fourgram"].apply(", ".join).reset_index()
df_word_order_join_fivegram = df_word_order_fivegram.groupby(["word"])["fivegram"].apply(", ".join).reset_index()
df_word_order_join_sentence = df_word_order_sentence.groupby(["word"])["sentence"].apply(", ".join).reset_index()
df_word_order_join_twogram_threegram = df_word_order_twogram_threegram.groupby(["word"])["twogram_in_threegram"].apply(", ".join).reset_index()

In [24]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram,df_word_order_join_fourgram,df_word_order_join_fivegram,df_word_order_join_sentence,df_word_order_join_twogram_threegram]

In [25]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all  

,word,twogram,threegram,fourgram,fivegram,sentence,twogram_in_threegram
0,abaküs,bir abaküs,NaN,NaN,NaN,bu bir abaküs,NaN
1,abanoz,"bu abanoz, stres abanoz, için abanoz, abanoz bunu",NaN,NaN,NaN,doktor bu abanoz,NaN
2,abluka,bir abluka,NaN,NaN,NaN,"bu bir abluka, abluka mı",NaN
3,abrakadabra,"abrakadabra de, abrakadabra ve",NaN,NaN,NaN,ve abrakadabra,NaN
4,absorbe,onu absorbe,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1659,pik,NaN,NaN,NaN,NaN,bir pik,NaN
1660,rulet,NaN,NaN,NaN,NaN,"rulet mi, rulet gibi",NaN
1661,tekniker,NaN,NaN,NaN,NaN,bir tekniker,NaN
1662,tork,NaN,NaN,NaN,NaN,tork var,NaN


In [26]:
# Option For Word Frequency
if shared_word_frequency:
    df_word_order_join_all = pd.merge(df_word_order_join_all,df_word_all, how="inner", on="word")
    df_word_order_join_all.drop_duplicates(inplace=True)
    df_word_order_join_all = df_word_order_join_all.loc[:,["word","frequency","twogram","threegram","fourgram","fivegram","sentence","twogram_in_threegram"]]
    df_word_order_join_all.sort_values(by="frequency", inplace=True, ascending=False)
    df_word_order_join_all.reset_index(drop=True, inplace=True)
else:
    pass

df_word_order_join_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence,twogram_in_threegram
0,bir,18835735,"çek bir, bir petrol, bir sperm, bir stres, müs...","bir şey yok, bir şey var, bir şey değil, bir ş...","bir şey var mı, bir şey daha var, bir şey mi v...","komik bir şey mi var, bir şey yok değil mi, bi...","bir şey değil, yok bir şey, bir şey yok, bir ş...","bir şey, bu bir, bir daha, ben bir, bana bir, ..."
1,bu,11062659,"bu asansör, bu kaptan, bu park, bu profesör, b...","bu da ne, bu kadar mı, ve bu da, bu kadar çok,...","bu şey de ne, bu o kadar da, bir şey değil bu,...","bu da bir şey mi, bu daha bir şey değil, bu no...","bu da ne, bu ne, bu o, bu kadar, bu kadar mı, ...","bu kadar, bu da, bu çok, bu bir, bu ne, ama bu..."
2,ne,8025880,"şifre ne, maç ne, tuvalet ne, şerif ne, ne tak...","bu da ne, ne kadar da, o da ne, daha ne kadar,...","ne var ne yok, bu şey de ne, daha ne kadar var...","ne kadar çok o kadar, ne sen ne de ben, ne kad...","ne var, bu da ne, ne için, bu ne, ne kadar, ne...","ne kadar, ne var, da ne, bu ne, sen ne, ne içi..."
3,ve,7766036,"ve motor, ve film, ve disiplin, ve gol, ve par...","sen ve ben, ve ben de, ve sen de, ve bu da, ve...","ve bir şey daha, ve sen de bunu, ve ben de onu...","ve bir şey daha var, ve bir o kadar da, ve bir...","sen ve ben, ve sen, ve ben, ve bu, ve ne, ve b...","ve ben, ve bu, ve bir, sen ve, ve sen, ve o, v..."
4,için,5484109,"için kontrol, için polis, için onur, bomba içi...","için bir şey, için ne kadar, bir şey için, içi...","için bir şey var, için çok ama çok, için bana ...","için bir şey var mı, için bir şey mi var, için...","ne için, bu ne için, o ne için, şey için, ne i...","için bir, için çok, ne için, için mi, için ne,..."
...,...,...,...,...,...,...,...,...
1659,roma,15,roma sen,NaN,NaN,NaN,NaN,NaN
1660,refüze,13,seni refüze,NaN,NaN,NaN,NaN,NaN
1661,planya,12,bir planya,NaN,NaN,NaN,NaN,NaN
1662,epidemi,8,bir epidemi,NaN,NaN,NaN,NaN,NaN


In [27]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency{file_ext}2.xlsx", index=False)

#### Copy Move And Delete

In [28]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*{file_ext}2.xlsx")
output_file

['Turkish_English_Shared_Result_With_Frequency12.xlsx',
 'Turkish_English_Shared_Join_Result_Without_Frequency12.xlsx']

In [29]:
for k in output_file:
    source = k # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [30]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

### Prefix Suffix Result Analysis

In [19]:
import os
import multiprocessing
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # 28  # native word end index

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = True  # always must be True in this part
native_word = False # True for adding native word
etymology_word = True  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.
print(f"{file_ext}")

5


In [3]:
def word_in_wordgroup(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column. target_column values occur
       source_column's words
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            #word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(10)  # Option
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_word_result = pd.concat([df_word_result,word_in_word_cluster], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [4]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
989326,karneleme,5
989327,karnaya,5
989328,dörtlümüzün,5
989329,karnavalınız,5


In [6]:
df_word_prefix_suffix = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
df_word_prefix_suffix = df_word_prefix_suffix[["search_word","word"]]
df_word_prefix_suffix

,search_word,word
0,ama,ama
1,bana,bana
2,ben,ben
3,ben,bence
4,ben,bende
...,...,...
162,şey,şeyler
163,şey,şeylerden
164,şey,şeylere
165,şey,şeyleri


In [8]:
df_ety_prefix_suffix = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
df_ety_prefix_suffix = df_ety_prefix_suffix[["search_word","word"]]
df_ety_prefix_suffix

,search_word,word
0,abaküs,abaküs
1,abandone,abandoned
2,abanoz,abanoz
3,abdomen,abdomende
4,abluka,abluka
...,...,...
6188,şut,şutu
6189,şut,şutunu
6190,şırınga,şırınga
6191,şırınga,şırıngayla


In [9]:
if file_ext == "3":
    df_all_word = pd.concat([df_word_prefix_suffix,df_ety_prefix_suffix],axis=0)
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
elif file_ext == "4":
    df_all_word = df_ety_prefix_suffix
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
elif file_ext == "5":
    df_all_word = df_word_prefix_suffix
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
else:
    pass
    
df_all_word

,search_word,word
0,ama,ama
1,bana,bana
2,ben,ben
3,ben,bence
4,ben,bende
...,...,...
162,şey,şeyler
163,şey,şeylerden
164,şey,şeylere
165,şey,şeyleri


In [10]:
df_all_word.search_word.nunique()

28

In [11]:
df_all_word.word.nunique()

155

In [35]:
df_shared_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency{file_ext}.xlsx")
#df_shared_all = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency.xlsx")
df_shared_all

,word,frequency,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,bana,2659182.0,bana biraz,9044,milyon dolardan fazla,279.0,kredi kartları sivil frekans,23.0,telefon santrali postane elektrik santrali,12.0,telefon numarası,232.0
1,biraz,1269641.0,milyon dolar,6377,standart kablolu orkestrası,191.0,fazla kaliteli normal banka,20.0,cd radyo televizyon telefon bar,10.0,biraz komik,229.0
2,fazla,692043.0,biraz fazla,5111,bana biraz şeker,176.0,servis modülü reaksiyon kontrol,20.0,radyo televizyon telefon bar video,10.0,kredi kartı,208.0
3,asla,463224.0,bana şans,2722,bana telefon numaranı,144.0,müslüman türkler avrupada hıristiyanlık,17.0,servis modülü reaksiyon kontrol sistemi,10.0,standart prosedür,192.0
4,kontrol,272328.0,kredi kartı,1940,bana biraz fazla,118.0,sistemleri telekom sektöründe lider,17.0,modülü reaksiyon kontrol sistemi valfleri,10.0,sezon finali,155.0
...,...,...,...,...,...,...,...,...,...,...,...,...
11156,NaN,NaN,depresif partisine,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11157,NaN,NaN,sistem stabil,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11158,NaN,NaN,sistem telefon,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11159,NaN,NaN,ekrandaki formüle,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
df_word_order_twogram = word_in_wordgroup(df_shared_all, "word", "twogram")
df_word_order_threegram = word_in_wordgroup(df_shared_all, "word", "threegram") 
df_word_order_fourgram = word_in_wordgroup(df_shared_all, "word", "fourgram") 
df_word_order_fivegram = word_in_wordgroup(df_shared_all, "word", "fivegram")
df_word_order_sentence = word_in_wordgroup(df_shared_all, "word", "sentence") 

In [38]:
df_word_order_twogram = pd.merge(df_word_order_twogram,df_all_word, how="inner", on="word")
df_word_order_threegram = pd.merge(df_word_order_threegram,df_all_word, how="inner", on="word")
df_word_order_fourgram = pd.merge(df_word_order_fourgram,df_all_word, how="inner", on="word")
df_word_order_fivegram = pd.merge(df_word_order_fivegram,df_all_word, how="inner", on="word")
df_word_order_sentence = pd.merge(df_word_order_sentence,df_all_word, how="inner", on="word")

In [39]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["search_word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["search_word"])["threegram"].apply(", ".join).reset_index()
df_word_order_join_fourgram = df_word_order_fourgram.groupby(["search_word"])["fourgram"].apply(", ".join).reset_index()
df_word_order_join_fivegram = df_word_order_fivegram.groupby(["search_word"])["fivegram"].apply(", ".join).reset_index()
df_word_order_join_sentence = df_word_order_sentence.groupby(["search_word"])["sentence"].apply(", ".join).reset_index()

In [40]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram,df_word_order_join_fourgram,df_word_order_join_fivegram,df_word_order_join_sentence]

In [41]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['search_word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all.rename(columns={"search_word":"word"}, inplace=True)
df_word_order_join_all

,word,twogram,threegram,fourgram,fivegram,sentence
0,abanoz,stres abanoz,NaN,NaN,NaN,NaN
1,abluka,"abluka riskini, asla ablukayı, destroyeri ablu...",NaN,NaN,NaN,NaN
2,absorbe,radyasyonu absorbe,NaN,NaN,NaN,NaN
3,acente,acente kopyası,NaN,NaN,NaN,NaN
4,adaptasyon,atmosfere adaptasyon,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1420,polimer,NaN,NaN,NaN,NaN,karbon polimer
1421,sedan,NaN,NaN,NaN,NaN,gri sedan
1422,sübjektif,NaN,NaN,NaN,NaN,biraz sübjektif
1423,termometre,NaN,NaN,NaN,NaN,"diferansiyel termometre, anüs termometresi"


In [44]:
# Option For Word Frequency
if shared_word_frequency:
    df_word_order_join_all = pd.merge(df_word_order_join_all,df_word_all, how="left", on="word")
    df_word_order_join_all.drop_duplicates(inplace=True)
    df_word_order_join_all = df_word_order_join_all.loc[:,["word","frequency","twogram","threegram","fourgram","fivegram","sentence"]]
    df_word_order_join_all.sort_values(by="frequency", inplace=True, ascending=False)
    df_word_order_join_all.reset_index(drop=True, inplace=True)
else:
    pass

df_word_order_join_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence
0,kontrol,272328.0,"numarayı kontrol, trafik kontrol, asla kontrol...","reaksiyon kontrol sistemi, kredi kartını kontr...","servis modülü reaksiyon kontrol, modülü reaksi...","servis modülü reaksiyon kontrol sistemi, modül...","rutin kontrol, manuel kontrol, mikrofon kontro..."
1,doktor,266714.0,"doktor bana, bana doktor, doktor fane, şanslar...","doktor bazı testler, ambulans avukat doktor, d...","şampiyonu doktor üniversite profesörü, olimpiy...",NaN,"doktor fane, pardon doktor, doktor park, bravo..."
2,polis,247969.0,"polis şefi, polis departmanı, polis bana, fazl...","metro polis departmanı, polis robot polis, met...","polis istasyonunda alarm otomatik, bazı polisl...",NaN,"alo polis, polis şefi, polis departmanı, sivil..."
3,dolar,199343.0,"milyon dolar, milyonlarca dolar, dolar avans, ...","dolar biraz fazla, amerika milyonlarca dolar, ...",NaN,NaN,"milyonlarca dolar, milyon dolar, seksen dolar,..."
4,vay,162887.0,"vay rekor, vay termostat, vay polis, vay süper...",NaN,NaN,NaN,"vay süper, vay çakal, vay bana, vay doktor"
...,...,...,...,...,...,...,...
1420,barmen,NaN,"barmenlik kursu, barmenlik sertifikası, barmen...",NaN,NaN,NaN,NaN
1421,ekolayzer,NaN,ekolayzerin modern,NaN,NaN,NaN,NaN
1422,metres,NaN,"metresinin metresi, fransız metresi, metresi m...",NaN,NaN,NaN,robotun metresi
1423,teras,NaN,"terasta latin, terastan terasa, terası kontrol...",NaN,NaN,NaN,NaN


In [259]:
df_word_order_join_all.word.nunique()

1425

In [260]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency{file_ext}3.xlsx", index=False)

#### Copy Move And Delete

In [261]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*{file_ext}3.xlsx")
output_file2

['Turkish_English_Shared_Join_Result_Without_Frequency43.xlsx']

In [262]:
for l in output_file2:
    source = l # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [263]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass

### Word Count Analysis

In [45]:
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil

In [46]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

In [47]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/Deploy Result Manuel").mkdir(parents=True, exist_ok=True)

In [ ]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

#### Hybrid Words

In [63]:
df_hybrid = pd.read_excel("Turkish English manual selected 2 gram hybrids 3.xlsx", sheet_name="2 gram hybrid")
df_hybrid

,word,twogram_pair1,twogram_pair2,twogram_pair3,twogram_pair4
0,bir,bir doktor,bir polis,bir dolar,bir telefon
1,bu,bu doktor,bu komik,bu telefon,bu bebek
2,ne,ne şans,mesaj ne,plan ne,ne sürpriz
3,ve,ve müzik,ve şeker,ve patates,ve çikolata
4,için,kontrol için,bebek için,dans için,film için
5,mi,komik mi,normal mi,problem mi,profesyonel mi
6,o,o ajan,o müzik,o park,o film
7,ben,ben senatör,ben ajan,ben general,ben profesör
8,de,patates de,şef de,profesör de,prenses de
9,çok,çok komik,çok normal,çok pardon,çok süper


In [64]:
df_hybrid_count = word_count_result(df_hybrid, ["twogram_pair1","twogram_pair2","twogram_pair3","twogram_pair4"])
df_hybrid_count

,word,word_count
0,bebek,6
1,komik,6
2,normal,5
3,sen,4
4,onu,4
...,...,...
89,okula,1
90,park,1
91,parka,1
92,pizza,1


In [65]:
df_hybrid_count_merge = pd.merge(df_hybrid,df_hybrid_count,how="left",on="word")
df_hybrid_count_merge

,word,twogram_pair1,twogram_pair2,twogram_pair3,twogram_pair4,word_count
0,bir,bir doktor,bir polis,bir dolar,bir telefon,4
1,bu,bu doktor,bu komik,bu telefon,bu bebek,4
2,ne,ne şans,mesaj ne,plan ne,ne sürpriz,4
3,ve,ve müzik,ve şeker,ve patates,ve çikolata,4
4,için,kontrol için,bebek için,dans için,film için,4
5,mi,komik mi,normal mi,problem mi,profesyonel mi,4
6,o,o ajan,o müzik,o park,o film,4
7,ben,ben senatör,ben ajan,ben general,ben profesör,4
8,de,patates de,şef de,profesör de,prenses de,4
9,çok,çok komik,çok normal,çok pardon,çok süper,4


In [67]:
df_hybrid_count_merge2 = pd.merge(df_hybrid,df_hybrid_count,how="outer",on="word")
df_hybrid_count_merge2

,word,twogram_pair1,twogram_pair2,twogram_pair3,twogram_pair4,word_count
0,bir,bir doktor,bir polis,bir dolar,bir telefon,4
1,bu,bu doktor,bu komik,bu telefon,bu bebek,4
2,ne,ne şans,mesaj ne,plan ne,ne sürpriz,4
3,ve,ve müzik,ve şeker,ve patates,ve çikolata,4
4,için,kontrol için,bebek için,dans için,film için,4
...,...,...,...,...,...,...
89,okula,NaN,NaN,NaN,NaN,1
90,park,NaN,NaN,NaN,NaN,1
91,parka,NaN,NaN,NaN,NaN,1
92,pizza,NaN,NaN,NaN,NaN,1


In [73]:
writer = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_Hybrid_Word_Count.xlsx", engine='xlsxwriter')

In [74]:
df_hybrid_count_merge.to_excel(writer, sheet_name='28_Hybrid_Word_Count', index=False)
df_hybrid_count_merge2.to_excel(writer, sheet_name='All_Hybrid_Word_Count', index=False)

In [75]:
writer.save()

#### Target Words

In [69]:
df_target = pd.read_excel("Turkish English manual selected 2 gram hybrids 3.xlsx", sheet_name="2 gram target")
df_target

,word,twogram_1,twogram_2,twogram_3,twogram_4
0,bir,bir şey,bir daha,bir de,bir çok
1,bu,bu kadar,bu ne,bu çok,bu değil
2,ne,ne kadar,ne için,bana ne,ne gibi
3,ve,ve de,ve evet,ve ne,ve sen
4,için,ne için,benim için,senin için,onun için
5,mi,değil mi,ben mi,sen mi,evet mi
6,o,o kadar,o da,o değil,evet o
7,ben,ben de,ben değil,hayır ben,ben mi
8,de,sen de,bir de,beni de,beni de
9,çok,bu çok,daha çok,çok var,çok şey


In [70]:
df_target_count = word_count_result(df_target, ["twogram_1","twogram_2","twogram_3","twogram_4"])
df_target_count

,word,word_count
0,evet,14
1,değil,14
2,hayır,13
3,bu,12
4,ne,11
5,o,10
6,de,10
7,daha,9
8,var,9
9,bana,8


In [71]:
df_target_count_merge = pd.merge(df_target,df_target_count,how="left",on="word")
df_target_count_merge

,word,twogram_1,twogram_2,twogram_3,twogram_4,word_count
0,bir,bir şey,bir daha,bir de,bir çok,7
1,bu,bu kadar,bu ne,bu çok,bu değil,12
2,ne,ne kadar,ne için,bana ne,ne gibi,11
3,ve,ve de,ve evet,ve ne,ve sen,5
4,için,ne için,benim için,senin için,onun için,6
5,mi,değil mi,ben mi,sen mi,evet mi,8
6,o,o kadar,o da,o değil,evet o,10
7,ben,ben de,ben değil,hayır ben,ben mi,7
8,de,sen de,bir de,beni de,beni de,10
9,çok,bu çok,daha çok,çok var,çok şey,7


In [72]:
df_target_count_merge2 = pd.merge(df_target,df_target_count,how="outer",on="word")
df_target_count_merge2

,word,twogram_1,twogram_2,twogram_3,twogram_4,word_count
0,bir,bir şey,bir daha,bir de,bir çok,7
1,bu,bu kadar,bu ne,bu çok,bu değil,12
2,ne,ne kadar,ne için,bana ne,ne gibi,11
3,ve,ve de,ve evet,ve ne,ve sen,5
4,için,ne için,benim için,senin için,onun için,6
5,mi,değil mi,ben mi,sen mi,evet mi,8
6,o,o kadar,o da,o değil,evet o,10
7,ben,ben de,ben değil,hayır ben,ben mi,7
8,de,sen de,bir de,beni de,beni de,10
9,çok,bu çok,daha çok,çok var,çok şey,7


In [76]:
writer2 = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_Target_Word_Count.xlsx", engine='xlsxwriter')

In [77]:
df_target_count_merge.to_excel(writer2, sheet_name='28_Target_Word_Count', index=False)
df_target_count_merge2.to_excel(writer2, sheet_name='All_Target_Word_Count', index=False)

In [78]:
writer2.save()

#### All Target Hybrid Word Count 

In [79]:
df_target_count

,word,word_count
0,evet,14
1,değil,14
2,hayır,13
3,bu,12
4,ne,11
5,o,10
6,de,10
7,daha,9
8,var,9
9,bana,8


In [80]:
df_hybrid_count

,word,word_count
0,bebek,6
1,komik,6
2,normal,5
3,sen,4
4,onu,4
...,...,...
89,okula,1
90,park,1
91,parka,1
92,pizza,1


In [82]:
df_all_word = pd.concat([df_target_count, df_hybrid_count], axis=0)
df_all_word

,word,word_count
0,evet,14
1,değil,14
2,hayır,13
3,bu,12
4,ne,11
...,...,...
89,okula,1
90,park,1
91,parka,1
92,pizza,1


In [87]:
df_all_word.groupby("word")[["word_count"]].sum().reset_index(inplace=True)
df_all_word.sort_values(by="word_count", ascending=False, inplace=True)
df_all_word

,word,word_count
0,evet,14
1,değil,14
2,hayır,13
3,bu,12
4,ne,11
...,...,...
64,süper,1
65,problem,1
66,manyaksın,1
67,popüler,1


In [89]:
df_all_word.to_excel(f"{lang_folder}_{lang_pair}_Target_Hybrid_Word_Count.xlsx", index=False)

#### Copy Move And Delete

In [4]:
output_file3 = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_*_Word_Count.xlsx")
output_file3

['Turkish_English_Hybrid_Word_Count.xlsx',
 'Turkish_English_Target_Word_Count.xlsx',
 'Turkish_English_Target_Hybrid_Word_Count.xlsx']

In [7]:
for m in output_file3:
    source = m # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [8]:
for n in output_file3:
    try:
        os.remove(n)
    except:
        pass